## **Imports**

In [1202]:
import pandas as pd
import numpy as np
import importlib
from application.modeling import disbursements
from application.modeling import interest_income
from application.modeling import other_income
from application.modeling import helper
from application.modeling import depreciation
from application.modeling import expenses
from application.modeling import income_statement
from application.modeling import direct_cashflow
from application.modeling import balance_sheet
from application.modeling import statement_of_cashflows
from application.modeling import loan_book

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**

In [1203]:
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12
IMTT = 0.01

In [1204]:
parameters = pd.read_csv("./data_templates/parameters.csv", index_col=0)
parameters.columns = pd.period_range(VALUATION_DATE,periods=int(parameters.columns[-1]), freq="M")
parameters.head()

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
Current Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NEW_INVENTORY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
INVENTORY_USED,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_INTERGROUP_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_PAYABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [1205]:
details_of_existing_assets = pd.read_csv("./data_templates/details_of_existing_assets.csv")
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,straight_line
2,Computers,0.20,10000,5,20000,10,straight_line
3,Furniture,0.10,50000,20,80000,25,straight_line


In [1206]:
details_of_new_assets = pd.read_csv("./data_templates/details_of_new_assets.csv")
details_of_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost
0,Vehicles,0.25,10000,5,straight_line,01/06/2023,10000
1,Buildings,0.10,50000,20,straight_line,01/09/2023,50000


In [1207]:
details_of_existing_long_term_borrowing = pd.read_csv("./data_templates/details_of_existing_long_term_borrowing.csv")
details_of_existing_long_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line
2,Grandre,160000,0.06,36,36,straight_line
3,Minerva,15000,0.08,36,36,straight_line
4,Masawara,5613,0.06,60,60,straight_line
5,Zam-Cargo,364000,0.05,36,36,straight_line


In [1208]:
details_of_new_long_term_borrowing = pd.read_csv("./data_templates/details_of_new_long_term_borrowing.csv")
details_of_new_long_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,60,60,01/07/2023,straight_line
1,ZAM,400000,0.25,60,60,01/07/2023,reducing_balance
2,NBS,500000,0.18,60,60,01/09/2023,straight_line
3,ZB,500000,0.21,60,60,01/10/2023,straight_line


In [1209]:
details_of_new_short_term_borrowing = pd.read_csv("./data_templates/details_of_new_short_term_borrowing.csv")
details_of_new_short_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [1210]:
details_of_existing_short_term_borrowing = pd.read_csv("./data_templates/details_of_existing_short_term_borrowing.csv")
details_of_existing_short_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Life,50000,0.14,10,12,straight_line
1,Zimnat Life,30000,0.13,12,12,straight_line
2,Grandre,100000,0.12,7,12,straight_line
3,FCB,120383,0.18,5,12,straight_line
4,Nurture,500000,0.25,8,12,straight_line
5,NMB,450000,0.22,11,12,straight_line
6,Beramasamba,30000,0.13,3,12,straight_line
7,Dandemutande,25000,0.12,7,12,reducing_balance


In [1211]:
static_inputs_income_statement = pd.read_csv("./data_templates/static_inputs_income_statement.csv", index_col=0)
static_inputs_income_statement

,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23
CILL,13152,11585,13943,17555,13073,11021,13843,17989,19692,16873,15675,10161
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878
Retrenchments,19268,14887,14859,16331,18571,18684,17208,15276,12062,10064,18006,12568
Staff Welfare,15463,12027,12695,19687,15258,15618,16736,10391,15892,13561,16184,13099
Donations,16278,18392,13104,17215,12454,18996,12731,18154,19762,15056,18110,13840
Legal Fees,0,0,0,0,0,0,0,0,0,0,0,0
Strategic Expenses,0,0,0,0,0,0,0,0,0,0,0,0
Rental Income,0,0,0,0,0,0,0,0,0,0,0,0
Interest Received,0,0,0,0,0,0,0,0,0,0,0,0
Dividends Received,0,0,0,0,0,0,0,0,0,0,0,0


In [1212]:
variable_inputs_income_statement = pd.read_csv("./data_templates/variable_inputs_income_statement.csv", index_col=0)
variable_inputs_income_statement

,Aug-22,Sep-22,Oct-22,Nov-22,Dec-22,Jan-23
Training,6028,5502,9493,5870,9488,5206
Travel Costs,6484,5863,7790,5563,9191,6757
Entertainment,6678,8242,9636,6059,6722,8314
Marketing Costs,8157,7625,7729,6597,8060,7693
Group Marketing Costs,9891,8627,6363,6981,6663,6529
Rental Costs,7038,8302,7237,6306,9029,7675
Subscriptions,6282,5709,9744,9413,8748,9779
Insurance,5663,6998,8445,8743,6495,8304
Repairs And Maintenance,8763,6853,9659,6291,8581,8457
Utilities,6636,8696,7999,8152,5698,7160


In [1213]:
opening_balances = pd.read_csv("./data_templates/opening_balances.csv")
opening_balances

,TRADE_RECEIVABLES,TRADE_PAYABLES,LOAN_BOOK,CASH_ON_HAND,ISSUED_SHARE_CAPITAL,SHARE_PREMIUM,OTHER_COMPONENTS_OF_EQUITY,TREASURY_SHARES,RETAINED_EARNINGS,LONG_TERM_LOANS,SHORT_TERM_LOANS,INTERCOMPANY_LOANS,DEFERED_TAXATION,OTHER_RECEIVABLES,OTHER_PAYABLES,PROVISION_FOR_TAXATION,PROVISION_FOR_CREDIT_LOSS,INVESTMENT_IN_SUBSIDIARIES,INVESTMENT_IN_ASSOCIATES,INVESTMENT_PROPERTIES,EQUITY_INVESTMENTS,LONG_TERM_MONEY_MARKET_INVESTMENTS,LOANS_TO_RELATED_ENTITIES,TREASURY_SHARES.1,INTANGIBLE_ASSETS,INVENTORIES,OTHER_RECEIVABLES.1,INTERGROUP_RECEIVABLES,SHORT_TERM_MONEY_MARKET_INVESTMENTS
0,1000000,940000,2800000,331000,7000,110,0,0,200000,200000,50000,0,0,0,0,0,10000,0,10000,0,20000,0,0,0,0,0,0,0,0


In [1214]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv", index_col=0)
existing_loans

,disbursement_date,maturity_date,loan_amount,frequency,loan_term_months,remaining_term_months,interest_rate,repayment_amount,outstanding_balance,admin_fee_percentage,credit_insurance_fee_percentage
loan_number,,,,,,,,,,,
40b0775c,43536,46386,2400,12,94,48,0.00,31.39,"1,351.78",0.06,0.05
f5c36555,43873,46386,4600,12,83,48,0.01,92.11,"3,246.86",0.06,0.07
75656bce,44068,46386,2400,12,77,48,0.01,47.81,"1,739.68",0.08,0.05
a7222f00,44611,46021,3800,12,47,36,0.00,83.95,"2,935.72",0.09,0.08
ad00cbf5,43747,46751,5000,12,99,60,0.00,52.09,"3,066.98",0.07,0.05
321ec75b,44137,46751,1700,12,86,60,0.00,24.25,"1,258.85",0.08,0.10
6747e1d4,43606,46021,1600,12,80,36,0.01,31.43,911.36,0.05,0.10
5131c13c,44591,46386,1000,12,60,48,0.01,20.90,835.18,0.05,0.08
a7222f00_1,43520,46021,1200,12,83,36,0.01,18.50,595.83,0.06,0.05


## **New Disbursements**

In [1215]:
new_disbursements_df  = disbursements.calculate_new_disbursements(parameters=parameters)
new_disbursements_df

,b2b_disbursements,sme_disbursements,consumer_ssb_disbursements,consumer_pvt_disbursements,total
2023-01,"10,000.00","360,000.00","48,000.00","60,000.00","490,000.00"
2023-02,"10,000.00","432,000.00","60,000.00","75,000.00","592,000.00"
2023-03,"10,000.00","480,000.00","72,000.00","108,000.00","706,000.00"
2023-04,"10,000.00","360,000.00","80,000.00","120,000.00","610,000.00"
2023-05,"10,000.00","324,000.00","96,000.00","144,000.00","622,000.00"
2023-06,"10,000.00","378,000.00","96,000.00","162,000.00","712,000.00"
2023-07,"10,000.00","420,000.00","112,000.00","189,000.00","808,000.00"
2023-08,"10,000.00","595,000.00","120,000.00","202,500.00","1,010,000.00"
2023-09,"10,000.00","700,000.00","120,000.00","225,000.00","1,160,000.00"
2023-10,"10,000.00","990,000.00","120,000.00","225,000.00","1,450,000.00"


### **Calculate Repayment Amount For New Disbursements**

In [1216]:
monthly_repayment_new_disbursements_df = interest_income.calculate_monthly_repayments_new_disbursements(new_disbursements_df=new_disbursements_df, parameters=parameters)
monthly_repayment_new_disbursements_df

,sme_monthly_repayment,b2b_monthly_repayment,consumer_ssb_monthly_repayment,consumer_pvt_monthly_repayment,total
2023-01,"81,451.18","1,627.45","8,006.34","10,007.93","101,092.91"
2023-02,"97,741.42","1,627.45","10,007.93","12,509.91","121,886.71"
2023-03,"108,601.58","1,627.45","12,009.51","18,014.27","140,252.81"
2023-04,"81,451.18","1,627.45","11,172.05","16,758.08","111,008.77"
2023-05,"73,306.06","1,627.45","13,406.46","20,109.69","108,449.68"
2023-06,"85,523.74","1,627.45","13,406.46","22,623.41","123,181.06"
2023-07,"95,026.38","1,627.45","15,640.87","26,393.97","138,688.68"
2023-08,"134,620.70","1,627.45","14,887.07","28,279.26","179,414.48"
2023-09,"158,377.30","1,627.45","14,887.07","31,421.40","206,313.22"
2023-10,"223,990.75","1,627.45","13,705.47","31,421.40","270,745.08"


### **Generate Loan Schedules For New Disbursements**

In [1217]:
loan_schedules_for_all_new_disbursements = (
    interest_income.generate_loan_schedules_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        monthly_repayment_new_disbursements_df=monthly_repayment_new_disbursements_df,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)
loan_schedules_for_all_new_disbursements.keys()

dict_keys(['sme_loan_schedules', 'b2b_loan_schedules', 'consumer_ssb_loan_schedules', 'consumer_pvt_loan_schedules'])

### **Capital Repayment New Disbursement**

In [1218]:
capital_repayment_new_disbursements_df = interest_income.generate_capital_repayment_new_disbursements_df(loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements)
capital_repayment_new_disbursements_df

,sme_capital_repayments,b2b_capital_repayments,consumer_pvt_capital_repayments,consumer_ssb_capital_repayments,total
Feb-2023,"47,251.18",627.45,"4,607.93","3,686.34","56,172.91"
Mar-2023,"108,441.46","1,317.65","10,782.55","8,626.04","129,167.71"
Apr-2023,"181,744.98","2,076.87","20,047.25","14,931.90","218,801.00"
May-2023,"246,261.93","2,912.01","27,809.58","20,247.82","297,231.35"
Jun-2023,"312,182.88","3,830.67","37,462.14","26,836.59","380,312.28"
Jul-2023,"391,453.99","4,841.19","48,877.14","34,018.35","479,190.67"
Aug-2023,"402,317.32","5,952.76","62,660.06","42,640.87","513,571.01"
Sep-2023,"420,891.75","7,175.49","78,353.72","50,565.62","556,986.58"
Oct-2023,"444,152.19","8,520.50","96,576.95","59,203.59","608,453.22"
Nov-2023,"534,836.21","10,000.00","106,432.35","59,431.04","710,699.61"


### **Interest Income New Disbursements**


In [1219]:
interest_income_new_disbursements_df = interest_income.generate_interest_income_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)

interest_income_new_disbursements_df

,sme_interest_income,b2b_interest_income,consumer_pvt_interest_income,consumer_ssb_interest_income,total
Feb-2023,"34,200.00","1,000.00","5,400.00","4,320.00","44,920.00"
Mar-2023,"70,751.14","1,937.25","11,735.29","9,388.23","93,811.91"
Apr-2023,"106,049.20","2,805.49","20,484.86","15,091.89","144,431.43"
May-2023,"122,983.43","3,597.80","29,480.60","20,948.01","177,009.85"
Jun-2023,"130,368.54","4,306.60","39,937.74","27,765.71","202,378.60"
Jul-2023,"136,621.17","4,923.53","51,146.15","33,990.42","226,681.27"
Aug-2023,"139,333.04","5,439.41","63,757.21","41,008.77","249,538.43"
Sep-2023,"157,637.89","5,844.14","76,342.80","47,971.09","287,795.92"
Oct-2023,"184,153.18","6,126.59","89,540.97","54,220.18","334,040.92"
Nov-2023,"236,008.72","6,274.54","101,099.04","59,691.86","403,074.16"


### **Calculate Admin Fee On New Disbursements**

In [1220]:
admin_fee_for_all_new_disbursements_df = other_income.calculate_admin_fee_for_all_new_disbursements(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
)

admin_fee_for_all_new_disbursements_df

,sme_admin_fee,b2b_admin_fee,consumer_ssb_admin_fee,consumer_pvt_admin_fee,total
Jan-2023,"3,000.00",10.00,184.62,272.73,"3,467.34"
Feb-2023,"6,600.00",20.00,415.38,613.64,"7,649.02"
Mar-2023,"10,600.00",30.00,692.31,"1,104.55","12,426.85"
Apr-2023,"13,600.00",40.00,"1,000.00","1,650.00","16,290.00"
May-2023,"16,300.00",50.00,"1,369.23","2,304.55","20,023.78"
Jun-2023,"19,450.00",60.00,"1,738.46","3,040.91","24,289.37"
Jul-2023,"19,950.00",70.00,"2,169.23","3,900.00","26,089.23"
Aug-2023,"21,308.33",80.00,"2,630.77","4,820.45","28,839.56"
Sep-2023,"23,141.67",90.00,"3,092.31","5,843.18","32,167.16"
Oct-2023,"28,391.67",100.00,"3,553.85","6,865.91","38,911.42"


### **Calculate Credit Life Insurance On New Disbursement**

In [1221]:
credit_insurance_fee_for_all_new_disbursements = other_income.calculate_credit_insurance_fee_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        months_to_forecast=MONTHS_TO_FORECAST,
    )

credit_insurance_fee_for_all_new_disbursements

,sme_credit_insurance_fee,b2b_credit_insurance_fee,consumer_ssb_credit_insurance_fee,consumer_pvt_credit_insurance_fee,total
Jan-2023,"1,200.00",0.00,73.85,109.09,"1,382.94"
Feb-2023,"2,640.00",0.00,166.15,245.45,"3,051.61"
Mar-2023,"4,240.00",0.00,276.92,441.82,"4,958.74"
Apr-2023,"5,440.00",0.00,400.00,660.00,"6,500.00"
May-2023,"6,520.00",0.00,547.69,921.82,"7,989.51"
Jun-2023,"7,780.00",0.00,695.38,"1,216.36","9,691.75"
Jul-2023,"7,980.00",0.00,867.69,"1,560.00","10,407.69"
Aug-2023,"8,523.33",0.00,"1,052.31","1,928.18","11,503.82"
Sep-2023,"9,256.67",0.00,"1,236.92","2,337.27","12,830.86"
Oct-2023,"11,356.67",0.00,"1,421.54","2,746.36","15,524.57"


### **Calculate For Credit Loss For New Disbursements**

In [1222]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",100.00,960.00,"1,800.00","10,060.00"
Feb-2023,"8,640.00",100.00,"1,200.00","2,250.00","12,190.00"
Mar-2023,"9,600.00",100.00,"1,440.00","3,240.00","14,380.00"
Apr-2023,"7,200.00",100.00,"1,600.00","3,600.00","12,500.00"
May-2023,"6,480.00",100.00,"1,920.00","4,320.00","12,820.00"
Jun-2023,"7,560.00",100.00,"1,920.00","4,860.00","14,440.00"
Jul-2023,"8,400.00",100.00,"2,240.00","5,670.00","16,410.00"
Aug-2023,"11,900.00",100.00,"2,400.00","6,075.00","20,475.00"
Sep-2023,"14,000.00",100.00,"2,400.00","6,750.00","23,250.00"
Oct-2023,"19,800.00",100.00,"2,400.00","6,750.00","29,050.00"


## **Existing Loans**

### **Generate Loan Schedules For Existing Loans**


In [1223]:
existing_loans_schedules = interest_income.generate_loan_schedules_existing_loans(
    outstanding_balance=existing_loans["outstanding_balance"],
    interest_rate_monthly=existing_loans["interest_rate"],
    repayment_amount_monthly=existing_loans["repayment_amount"],
    valuation_date=VALUATION_DATE, 
    months_to_project=MONTHS_TO_FORECAST,
)
existing_loans_schedules.keys()

dict_keys(['interest', 'capital_repayment', 'outstanding_at_start'])

### **Capital Repayment for Existing Loans**

In [1224]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayment'].sum()
capital_repayment_existing_loans

Jan-2023   884.89
Feb-2023   891.70
Mar-2023   898.59
Apr-2023   905.55
May-2023   912.59
Jun-2023   919.70
Jul-2023   926.89
Aug-2023   934.16
Sep-2023   941.52
Oct-2023   948.95
Nov-2023   956.47
Dec-2023     0.00
dtype: float64

### **Intererest Income For Existing Loans**

In [1225]:
interest_income_existing_loans = existing_loans_schedules['interest'].sum()
interest_income_existing_loans

Jan-2023   290.78
Feb-2023   283.97
Mar-2023   277.09
Apr-2023   270.13
May-2023   263.09
Jun-2023   255.98
Jul-2023   248.78
Aug-2023   241.51
Sep-2023   234.16
Oct-2023   226.73
Nov-2023   219.21
Dec-2023     0.00
dtype: float64

### **Calculate Other Income Existing Loans**

In [1226]:
other_income_existing_loans_df = other_income.calculate_other_income_existing_loans(existing_loans=existing_loans, valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST)
other_income_existing_loans_df

,credit_insurance_fee_existing_loans,admin_fee_existing_loans,total
Jan-2023,61.55,66.22,127.78
Feb-2023,61.55,66.22,127.78
Mar-2023,61.55,66.22,127.78
Apr-2023,61.55,66.22,127.78
May-2023,61.55,66.22,127.78
Jun-2023,61.55,66.22,127.78
Jul-2023,61.55,66.22,127.78
Aug-2023,61.55,66.22,127.78
Sep-2023,61.55,66.22,127.78
Oct-2023,61.55,66.22,127.78


## **Interest Income**

In [1227]:
total_interest_income = interest_income.aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements_df=interest_income_new_disbursements_df,
    interest_income_existing_loans= interest_income_existing_loans,
    valuation_date= VALUATION_DATE, 
    months_to_forecast= MONTHS_TO_FORECAST
)
total_interest_income

Jan-2023       290.78
Feb-2023    45,203.97
Mar-2023    94,088.99
Apr-2023   144,701.56
May-2023   177,272.94
Jun-2023   202,634.57
Jul-2023   226,930.05
Aug-2023   249,779.94
Sep-2023   288,030.08
Oct-2023   334,267.64
Nov-2023   403,293.37
Dec-2023   475,262.01
dtype: float64

## **Other Income**

In [1228]:
other_income_df = other_income.aggregate_other_income(
    admin_fee_for_all_new_disbursements_df=admin_fee_for_all_new_disbursements_df,
    admin_fee_existing_loans=other_income_existing_loans_df['admin_fee_existing_loans'],
    credit_insurance_fee_existing_loans=other_income_existing_loans_df['credit_insurance_fee_existing_loans'],
    credit_insurance_fee_for_all_new_disbursements_df=credit_insurance_fee_for_all_new_disbursements, 
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)
other_income_df

,admin_fee,credit_insurance_fee,total
Jan-2023,"7,000.91","2,827.43","9,828.34"
Feb-2023,"15,364.26","6,164.77","21,529.03"
Mar-2023,"24,919.93","9,979.04","34,898.96"
Apr-2023,"32,646.22","13,061.55","45,707.78"
May-2023,"40,113.77","16,040.57","56,154.35"
Jun-2023,"48,644.96","19,445.05","68,090.01"
Jul-2023,"52,244.68","20,876.94","73,121.62"
Aug-2023,"57,745.34","23,069.20","80,814.54"
Sep-2023,"64,400.53","25,723.28","90,123.81"
Oct-2023,"77,889.07","31,110.69","108,999.76"


## **Depreciation**

In [1229]:
importlib.reload(depreciation)

<module 'application.modeling.depreciation' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\depreciation.py'>

In [1230]:
depreciations_and_nbvs = depreciation.calculate_depreciations_and_nbvs(
    details_of_existing_assets=details_of_existing_assets,
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST
)
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [1231]:
depreciations_and_nbvs['nbvs']

,Buildings,Computers,Furniture,Vehicles,total
Jan-2023,"42,000.00","6,000.00","42,000.00","6,000.00","96,000.00"
Feb-2023,"34,000.00","2,000.00","34,000.00","2,000.00","72,000.00"
Mar-2023,"26,000.00","-2,000.00","26,000.00","-2,000.00","48,000.00"
Apr-2023,"18,000.00","-6,000.00","18,000.00","-6,000.00","24,000.00"
May-2023,"10,000.00","-10,000.00","10,000.00","-10,000.00",0.00
Jun-2023,"2,000.00",0.00,"2,000.00","7,500.00","11,500.00"
Jul-2023,"-6,000.00",0.00,"-6,000.00","5,000.00","-7,000.00"
Aug-2023,"-14,000.00",0.00,"-14,000.00","2,500.00","-25,500.00"
Sep-2023,"23,000.00",0.00,"-22,000.00",0.00,"1,000.00"
Oct-2023,"10,000.00",0.00,"-30,000.00","-2,500.00","-22,500.00"


## **Salaries**

In [1232]:
importlib.reload(expenses)

<module 'application.modeling.expenses' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\expenses.py'>

In [1233]:
salaries_and_pension_and_statutory_contributions_df  = expenses.calculate_salaries_and_pension_and_statutory_contributions(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)
salaries_and_pension_and_statutory_contributions_df

,agent_commission,credit_officer_salaries,credit_officer_commission,other_staff_salaries,total,pensions_and_statutory_contributions
Jan-2023,"5,400.00",600.00,"3,600.00","17,000.00","26,600.00","3,021.76"
Feb-2023,"6,750.00",600.00,"4,320.00","17,000.00","28,670.00","3,256.91"
Mar-2023,"9,000.00",600.00,"4,800.00","17,000.00","31,400.00","3,567.04"
Apr-2023,"10,000.00",600.00,"3,600.00","17,000.00","31,200.00","3,544.32"
May-2023,"12,000.00",900.00,"3,240.00","17,000.00","33,140.00","3,764.70"
Jun-2023,"12,900.00",900.00,"3,780.00","17,000.00","34,580.00","3,928.29"
Jul-2023,"15,050.00","1,200.00","4,200.00","17,000.00","37,450.00","4,254.32"
Aug-2023,"16,125.00","1,400.00","5,950.00","17,000.00","40,475.00","4,597.96"
Sep-2023,"17,250.00","1,400.00","7,000.00","17,000.00","42,650.00","4,845.04"
Oct-2023,"17,250.00","1,800.00","9,900.00","17,000.00","45,950.00","5,219.92"


## **Provisions**


In [1234]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",100.00,960.00,"1,800.00","10,060.00"
Feb-2023,"8,640.00",100.00,"1,200.00","2,250.00","12,190.00"
Mar-2023,"9,600.00",100.00,"1,440.00","3,240.00","14,380.00"
Apr-2023,"7,200.00",100.00,"1,600.00","3,600.00","12,500.00"
May-2023,"6,480.00",100.00,"1,920.00","4,320.00","12,820.00"
Jun-2023,"7,560.00",100.00,"1,920.00","4,860.00","14,440.00"
Jul-2023,"8,400.00",100.00,"2,240.00","5,670.00","16,410.00"
Aug-2023,"11,900.00",100.00,"2,400.00","6,075.00","20,475.00"
Sep-2023,"14,000.00",100.00,"2,400.00","6,750.00","23,250.00"
Oct-2023,"19,800.00",100.00,"2,400.00","6,750.00","29,050.00"


### **Calculating Finance Costs**

In [1235]:
finance_costs = expenses.calculate_finance_costs(
    details_of_existing_long_term_borrowing = details_of_existing_long_term_borrowing,
    details_of_existing_short_term_borrowing = details_of_existing_short_term_borrowing,
    details_of_new_short_term_borrowing = details_of_new_short_term_borrowing,
    details_of_new_long_term_borrowing =details_of_new_long_term_borrowing,
    valuation_date= VALUATION_DATE,
    months_to_forecast= MONTHS_TO_FORECAST,
)

finance_costs.loc['total']

Jan-2023   60,659.71
Feb-2023   60,659.71
Mar-2023   60,659.71
Apr-2023   60,334.71
May-2023   60,334.71
Jun-2023   58,528.97
Jul-2023   58,528.97
Aug-2023   57,278.97
Sep-2023   46,862.30
Oct-2023   46,862.30
Nov-2023   46,278.97
Dec-2023   38,028.97
Name: total, dtype: float64

## **Income Statement**

In [1236]:
income_statement_index = pd.read_csv("./data_templates/income_statement_index.csv")
income_statement_index

,INCOME_STATEMENT
0,Interest Income
1,Other Income
2,Total Revenue
3,MANAGEMENT EXPENSES
4,STAFF COSTS
5,Salaries
6,Pensions & Statutory Contributions
7,Training
8,Bonus Provision
9,Retrenchments


In [1237]:
income_statement_df = income_statement.generate_income_statement_template(
    income_statement_index=income_statement_index,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Variable Expenses**

In [1238]:
variable_expenses = expenses.calculate_variable_expenses(
    variable_inputs_income_statement=variable_inputs_income_statement,
    parameters=parameters,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
variable_expenses

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Travel Costs,"7,080.16","7,221.76","7,366.20","7,513.52","7,663.79","7,817.07","7,973.41","8,132.88","8,295.54","8,461.45","8,630.68","8,803.29"
Entertainment,"7,760.67","7,915.88","8,074.20","8,235.69","8,400.40","8,568.41","8,739.77","8,914.57","9,092.86","9,274.72","9,460.21","9,649.42"
Marketing Costs,"7,796.37","7,952.30","8,111.34","8,273.57","8,439.04","8,607.82","8,779.98","8,955.58","9,134.69","9,317.38","9,503.73","9,693.81"
Group Marketing Costs,"7,659.18","7,812.36","7,968.61","8,127.98","8,290.54","8,456.35","8,625.48","8,797.99","8,973.95","9,153.43","9,336.50","9,523.23"
Rental Costs,"7,749.79","7,904.79","8,062.88","8,224.14","8,388.62","8,556.39","8,727.52","8,902.07","9,080.11","9,261.72","9,446.95","9,635.89"
Subscriptions,"8,444.75","8,613.64","8,785.92","8,961.64","9,140.87","9,323.69","9,510.16","9,700.36","9,894.37","10,092.26","10,294.10","10,499.99"
Insurance,"7,590.16","7,741.96","7,896.80","8,054.74","8,215.83","8,380.15","8,547.75","8,718.71","8,893.08","9,070.94","9,252.36","9,437.41"
Repairs And Maintenance,"8,262.68","8,427.93","8,596.49","8,768.42","8,943.79","9,122.67","9,305.12","9,491.22","9,681.05","9,874.67","10,072.16","10,273.60"
Utilities,"7,537.97","7,688.73","7,842.50","7,999.35","8,159.34","8,322.53","8,488.98","8,658.76","8,831.93","9,008.57","9,188.74","9,372.52"


### **Calculate Change In Provision For Credit Loss**

In [1239]:
change_in_provision_for_credit_loss = (
    expenses.calculate_change_in_provision_for_credit_loss(
        provision_for_credit_loss=provision_for_credit_loss_for_all_new_disbursements['total'],
        provision_for_credit_loss_opening_balances=float(
            opening_balances["PROVISION_FOR_CREDIT_LOSS"].iat[0]
        ),
        valuation_date=VALUATION_DATE, 
        months_to_forecast=MONTHS_TO_FORECAST
    )
)

change_in_provision_for_credit_loss

Jan-2023       60.00
Feb-2023    2,130.00
Mar-2023    2,190.00
Apr-2023   -1,880.00
May-2023      320.00
Jun-2023    1,620.00
Jul-2023    1,970.00
Aug-2023    4,065.00
Sep-2023    2,775.00
Oct-2023    5,800.00
Nov-2023    2,700.00
Dec-2023        0.00
Name: total, dtype: float64

### **Calculate Business Acquisition**

In [1240]:
business_acquisition = expenses.calculate_business_acqusition(
    business_acquisition_percent=parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=new_disbursements_df['consumer_ssb_disbursements'],
    consumer_pvt_disbursements=new_disbursements_df['consumer_pvt_disbursements'],
)
business_acquisition

Jan-2023   1,620.00
Feb-2023   2,025.00
Mar-2023   2,484.00
Apr-2023   2,760.00
May-2023   3,312.00
Jun-2023   3,366.00
Jul-2023   3,927.00
Aug-2023   4,207.50
Sep-2023   4,275.00
Oct-2023   4,275.00
Nov-2023   4,275.00
Dec-2023   4,275.00
dtype: float64

### **Insert Revenue**

In [1241]:
income_statement_df = income_statement.insert_revenue(
    income_statement=income_statement_df,
    interest_income=total_interest_income,
    other_income=other_income_df['total'],
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","66,733.01","128,987.96","190,409.33","233,427.29","270,724.58","300,051.67","330,594.47","378,153.89","443,267.40","535,139.07","627,930.02"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Static And Variable Inputs**

In [1242]:
income_statement_df = income_statement.insert_static_and_variable_inputs(
    income_statement=income_statement_df,
    static_inputs_income_statement=static_inputs_income_statement, variable_expenses=variable_expenses
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","66,733.01","128,987.96","190,409.33","233,427.29","270,724.58","300,051.67","330,594.47","378,153.89","443,267.40","535,139.07","627,930.02"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Insert Salaries and Pensions And Statutory Contributions**

In [1243]:
income_statement_df = income_statement.insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement_df,
    salaries_and_pension_and_statutory_contributions_df = salaries_and_pension_and_statutory_contributions_df,
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","66,733.01","128,987.96","190,409.33","233,427.29","270,724.58","300,051.67","330,594.47","378,153.89","443,267.40","535,139.07","627,930.02"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


### **Insert Depreciation**

In [1244]:
income_statement_df = income_statement.insert_depreciation(
    income_statement=income_statement_df, depreciation=depreciations_and_nbvs['dpns']['total'])

### **Insert Change in Provision For Credit Loss**

In [1245]:
income_statement_df = income_statement.insert_credit_loss_provision(
    income_statement=income_statement_df, 
    change_in_provisin_for_credit_loss=change_in_provision_for_credit_loss,
)

In [1246]:
provision_for_credit_loss  = provision_for_credit_loss_for_all_new_disbursements['total']
provision_for_credit_loss

Jan-2023   10,060.00
Feb-2023   12,190.00
Mar-2023   14,380.00
Apr-2023   12,500.00
May-2023   12,820.00
Jun-2023   14,440.00
Jul-2023   16,410.00
Aug-2023   20,475.00
Sep-2023   23,250.00
Oct-2023   29,050.00
Nov-2023   31,750.00
Dec-2023   31,750.00
Name: total, dtype: float64

### **Insert Business Acquisition**

In [1247]:
income_statement_df = income_statement.insert_business_acquisition(
    income_statement=income_statement_df, business_acquisition=business_acquisition)

### **Aggregations**

In [1248]:
income_statement_df = income_statement.aggregate_staff_costs(income_statement_df)
income_statement_df = income_statement.aggregate_travel_and_entertainment(income_statement_df)
income_statement_df = income_statement.aggregate_marketing_and_public_relations(income_statement_df)
income_statement_df = income_statement.aggregate_office_costs(income_statement_df)
income_statement_df = income_statement.aggregate_professional_fees(income_statement_df)
income_statement_df = income_statement.aggregate_communication_costs(income_statement_df)
income_statement_df = income_statement.aggregate_motor_vehicle_costs(income_statement_df)
income_statement_df = income_statement.aggregate_other_costs(income_statement_df)
income_statement_df = income_statement.aggregate_investment_income(income_statement_df)
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculating Total Expenses and EBIDTA**

In [1249]:
income_statement_df = income_statement.calculate_total_expenses(income_statement_df)
income_statement_df = income_statement.calculate_ebidta(income_statement_df)

### **Insert Finance Cost**

In [1250]:
income_statement_df.loc['Finance Costs'] = finance_costs.loc['total']

### **Aggregate Finance Cost**

In [1251]:
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculate Profit Before Tax**

In [1252]:
income_statement_df = income_statement.calculate_profit_before_tax(income_statement_df)

### **Calculate Tax**

In [1253]:
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","66,733.01","128,987.96","190,409.33","233,427.29","270,724.58","300,051.67","330,594.47","378,153.89","443,267.40","535,139.07","627,930.02"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [1254]:
income_statement_df = income_statement.calculate_tax(income_statement=income_statement_df,tax_rate=parameters.loc['TAX_RATE'])
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","66,733.01","128,987.96","190,409.33","233,427.29","270,724.58","300,051.67","330,594.47","378,153.89","443,267.40","535,139.07","627,930.02"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


## **Direct Cashflow**

In [1355]:
direct_cashflow_df = direct_cashflow.generate_direct_cashflow_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Receipts From Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Purchase Of Inventory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Equity And Intercompany Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Purchase Of Other Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Available Direct Cashflow Items**

#### **From Parameters**

In [1356]:
direct_cashflow_df.loc[
    "Receipts From Receivables"
] = helper.change_period_index_to_strftime(
    helper.add_series(
        [
            parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
            parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
            parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
        ]
    )
)

direct_cashflow_df.loc[
    "Purchase Of Inventory"
] = helper.change_period_index_to_strftime(parameters.loc["NEW_INVENTORY"])
direct_cashflow_df.loc[
    "Payments To Payables"
] = -helper.change_period_index_to_strftime(
    helper.add_series(
        [
            parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
            parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
        ]
    )
)

direct_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    parameters.loc["DIVIDEND_PAID"]
)

#### **From Calculations/Income Statement**

In [1357]:
direct_cashflow_df.loc['Interest Income'] = income_statement_df.loc['Interest Income']
direct_cashflow_df.loc['Other Income'] = income_statement_df.loc['Other Income']
direct_cashflow_df.loc['Interest Expense'] = -income_statement_df.loc['Finance Costs']
direct_cashflow_df.loc['Disbursements'] = -helper.change_period_index_to_strftime(new_disbursements_df['total'])

In [1358]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Purchase Of Other Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Equity and Intercompany Loans**

In [1389]:
direct_cashflow_df = direct_cashflow.add_equity_and_intercompany_loans(
    parameters=parameters, direct_cashflow_df=direct_cashflow_df
)

### **Other Assets**


In [ ]:
direct_cashflow_df = direct_cashflow.add_other_assets(
    parameters=parameters, direct_cashflow_df=direct_cashflow_df
)

### **Tax Paid**

In [1366]:
tax_schedule = direct_cashflow.generate_tax_schedule(
    taxation=income_statement_df.loc["Taxation"],
    opening_balance=opening_balances["DEFERED_TAXATION"].iat[0],
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
tax_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"18,461.43"
Tax Charged,0,0,0,0,0,0,0,0,0,0,"18,461.43","49,646.86"
Tax Paid,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,"-68,108.28"
Closing Balance,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"18,461.43",0.00


In [1367]:
direct_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']

### **Calculate Operating Expenses**

In [1368]:
operating_expenses = direct_cashflow.calculate_operating_expenses(income_statement=income_statement_df)
operating_expenses

Jan-2023   298,736.02
Feb-2023   298,242.84
Mar-2023   303,599.96
Apr-2023   324,071.20
May-2023   319,667.16
Jun-2023   330,689.64
Jul-2023   335,135.60
Aug-2023   344,656.46
Sep-2023   357,415.36
Oct-2023   354,000.99
Nov-2023   372,785.39
Dec-2023   359,435.63
dtype: object

### **Insert Operating Expenses**

In [1369]:
direct_cashflow_df.loc['Operating Expenses']  = -operating_expenses
direct_cashflow_df


,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00",0.00,"1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


### **Calculate Capital Expenses**

In [1370]:
capital_expenses = direct_cashflow.calculate_capital_expenses(
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

capital_expenses

Jan-2023        0
Feb-2023        0
Mar-2023        0
Apr-2023        0
May-2023        0
Jun-2023    10000
Jul-2023        0
Aug-2023        0
Sep-2023    50000
Oct-2023        0
Nov-2023        0
Dec-2023        0
Name: cost, dtype: int64

### **Insert Capital Expenses**

In [1371]:
direct_cashflow_df.loc['Capital Expenses'] = -capital_expenses

### **Calculate Direct Cashflow Borrowing**

In [1372]:
long_and_short_term_borrowing_df = (
    direct_cashflow.calculate_long_and_short_term_borrowing_for_direct_cashflow(
        details_of_new_long_term_borrowing=details_of_new_long_term_borrowing,
        details_of_new_short_term_borrowing=details_of_new_short_term_borrowing,
        valuation_date=VALUATION_DATE,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)

long_and_short_term_borrowing_df

,long_term_borrowing,short_term_borrowing,total
Jan-2023,0,0,0
Feb-2023,0,0,0
Mar-2023,0,0,0
Apr-2023,0,0,0
May-2023,0,0,0
Jun-2023,0,0,0
Jul-2023,600000,600000,1200000
Aug-2023,0,0,0
Sep-2023,500000,500000,1000000
Oct-2023,500000,500000,1000000


### **Insert Direct Cashflow Borrowing**

In [1373]:
direct_cashflow_df.loc['Short Term Borrowing'] = long_and_short_term_borrowing_df['short_term_borrowing']
direct_cashflow_df.loc['Long Term Borrowing'] = long_and_short_term_borrowing_df['long_term_borrowing']

In [1374]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Long Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00",0.00,"1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


### **Calculate Capital Repayments**

In [1375]:
capital_repayment = helper.add_series(
[existing_loans_schedules['capital_repayment'].sum()
    , capital_repayment_new_disbursements_df['total']]
)

capital_repayment

Jan-2023       884.89
Feb-2023    57,064.61
Mar-2023   130,066.30
Apr-2023   219,706.55
May-2023   298,143.94
Jun-2023   381,231.98
Jul-2023   480,117.56
Aug-2023   514,505.17
Sep-2023   557,928.09
Oct-2023   609,402.17
Nov-2023   711,656.07
Dec-2023   842,349.66
Jan-2024   966,750.42
dtype: float64

### **Insert Capital Repayment**

In [1376]:
direct_cashflow_df.loc['Capital Repayment'] = capital_repayment

### **Calculate Capital Repayments on Borrowings**


In [1377]:
capital_repayment_on_borrowings_df = direct_cashflow.calculate_capital_repayment_on_borrowings(
        details_of_existing_long_term_borrowing=details_of_existing_long_term_borrowing,
        details_of_existing_short_term_borrowing=details_of_existing_short_term_borrowing,
        details_of_new_short_term_borrowing=details_of_new_short_term_borrowing,
        details_of_new_long_term_borrowing=details_of_new_long_term_borrowing,
        valuation_date=VALUATION_DATE,
        months_to_forecast=MONTHS_TO_FORECAST,
    )

capital_repayment_on_borrowings_df

,short_term_borrowing,long_term_borrowing,total
Jan-2023,0,0,0
Feb-2023,0,0,0
Mar-2023,0,0,0
Apr-2023,30000,0,30000
May-2023,0,0,0
Jun-2023,120383,0,120383
Jul-2023,0,0,0
Aug-2023,125000,0,125000
Sep-2023,500000,0,500000
Oct-2023,0,0,0


### **Insert Capital Repayment on New Borrowings**

In [1378]:
direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
] = -capital_repayment_on_borrowings_df['total']

### **Calculate Totals**

In [1379]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Long Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Capital Repayment,884.89,"57,064.61","130,066.30","219,706.55","298,143.94","381,231.98","480,117.56","514,505.17","557,928.09","609,402.17","711,656.07","842,349.66"
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00",0.00,"1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


In [1380]:
direct_cashflow_df.loc["Total Cash Inflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH INFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Inflows")
].sum()


direct_cashflow_df.loc["Total Cash Outflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH OUTFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Outflows")
].sum()

direct_cashflow_df.loc["Net Increase/Decrease In Cash"] = (
    direct_cashflow_df.loc["Total Cash Inflows"]
    + direct_cashflow_df.loc["Total Cash Outflows"]
)

### **Calculate Opening and Closing Balances**

In [1381]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Long Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Capital Repayment,884.89,"57,064.61","130,066.30","219,706.55","298,143.94","381,231.98","480,117.56","514,505.17","557,928.09","609,402.17","711,656.07","842,349.66"
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00",0.00,"1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


In [1383]:
direct_cashflow_df= direct_cashflow.calculate_opening_and_closing_balances_for_direct_cashflows(
direct_cashflow=direct_cashflow_df, 
cash_on_hand_opening_balance= opening_balances['CASH_ON_HAND'].iat[0]
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Long Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,"600,000.00",0.00,"500,000.00","500,000.00",0.00,0.00
Capital Repayment,884.89,"57,064.61","130,066.30","219,706.55","298,143.94","381,231.98","480,117.56","514,505.17","557,928.09","609,402.17","711,656.07","842,349.66"
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00",0.00,"1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


### **Calculate IMTT (2% Tax)**

In [1384]:
income_statement_df.loc['2% Taxation']  = -direct_cashflow_df.loc['Total Cash Outflows']*IMTT

## **Calculate Profit and Loss for the Period**


In [1385]:
income_statement_df = income_statement.calculate_profit_or_loss_for_period(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Other Income,"9,828.34","21,529.03","34,898.96","45,707.78","56,154.35","68,090.01","73,121.62","80,814.54","90,123.81","108,999.76","131,845.70","152,668.01"
Total Revenue,"10,119.12","66,733.01","128,987.96","190,409.33","233,427.29","270,724.58","300,051.67","330,594.47","378,153.89","443,267.40","535,139.07","627,930.02"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


## **Loan Book**

In [1153]:
loan_book_df = loan_book.generate_loan_book_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Loan Book Items**

In [1154]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayment'].sum()
capital_repayment_existing_loans

Jan-2023   884.89
Feb-2023   891.70
Mar-2023   898.59
Apr-2023   905.55
May-2023   912.59
Jun-2023   919.70
Jul-2023   926.89
Aug-2023   934.16
Sep-2023   941.52
Oct-2023   948.95
Nov-2023   956.47
Dec-2023     0.00
dtype: float64

In [1155]:
total_capital_repayments = loan_book.aggregate_new_and_existing_loans_capital_repayments(
    capital_repayments_new_disbursements_df = capital_repayment_new_disbursements_df,
    capital_repayments_existing_loans = capital_repayment_existing_loans,
    valuation_date = VALUATION_DATE,
    months_to_forecast = MONTHS_TO_FORECAST,
)
total_capital_repayments

Jan-2023       884.89
Feb-2023    57,064.61
Mar-2023   130,066.30
Apr-2023   219,706.55
May-2023   298,143.94
Jun-2023   381,231.98
Jul-2023   480,117.56
Aug-2023   514,505.17
Sep-2023   557,928.09
Oct-2023   609,402.17
Nov-2023   711,656.07
Dec-2023   842,349.66
dtype: float64

In [1156]:
loan_book_df = loan_book.insert_loan_book_items(
    loan_book=loan_book_df,
    opening_balance_on_loan_book=float(opening_balances["LOAN_BOOK"].iat[0]),
    total_interest_income=total_interest_income,
    total_capital_repayments=total_capital_repayments,
    disbursements=helper.change_period_index_to_strftime(new_disbursements_df["total"]),
)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,"490,000.00","592,000.00","706,000.00","610,000.00","622,000.00","712,000.00","808,000.00","1,010,000.00","1,160,000.00","1,450,000.00","1,585,000.00","1,585,000.00"
Repayments,"-1,175.68","-102,268.58","-224,155.29","-364,408.11","-475,416.87","-583,866.55","-707,047.61","-764,285.11","-845,958.18","-943,669.82","-1,114,949.44","-1,317,611.68"
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Opening and Closing Balances for Loan Book**

In [1157]:
loan_book_df = helper.calculate_opening_and_closing_balances(loan_book_df)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00","3,289,115.11","3,824,050.50","4,399,984.20","4,790,277.65","5,114,133.71","5,444,901.73","5,772,784.17","6,268,279.00","6,870,350.90","7,710,948.73","8,584,292.66"
New Disbursements,"490,000.00","592,000.00","706,000.00","610,000.00","622,000.00","712,000.00","808,000.00","1,010,000.00","1,160,000.00","1,450,000.00","1,585,000.00","1,585,000.00"
Repayments,"-1,175.68","-102,268.58","-224,155.29","-364,408.11","-475,416.87","-583,866.55","-707,047.61","-764,285.11","-845,958.18","-943,669.82","-1,114,949.44","-1,317,611.68"
Interest Income,290.78,"45,203.97","94,088.99","144,701.56","177,272.94","202,634.57","226,930.05","249,779.94","288,030.08","334,267.64","403,293.37","475,262.01"
Closing Balance,"3,289,115.11","3,824,050.50","4,399,984.20","4,790,277.65","5,114,133.71","5,444,901.73","5,772,784.17","6,268,279.00","6,870,350.90","7,710,948.73","8,584,292.66","9,326,942.99"


## **Balance Sheet**


In [1424]:
balance_sheet_df = balance_sheet.generate_balance_sheet_template(
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Available Balance Sheet Items**

In [1425]:
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [1426]:
balance_sheet_df.loc["Property Plant And Equipment"] = depreciations_and_nbvs["nbvs"][
    "total"
]

balance_sheet_df.loc["Loan Book"] = loan_book_df.loc["Closing Balance"]

balance_sheet_df.loc["Cash On Hand"] = direct_cashflow_df.loc["Closing Balance"]

balance_sheet_df.loc[
    "Provisions"
] = provision_for_credit_loss_for_all_new_disbursements["total"]


balance_sheet_df.loc["Provision For Taxation"] = helper.change_period_index_to_strftime(
    parameters.loc["PROVISION_FOR_TAX"]
)



balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"96,000.00","72,000.00","48,000.00","24,000.00",0.00,"11,500.00","-7,000.00","-25,500.00","1,000.00","-22,500.00","-41,000.00","-62,000.00"
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Generate Borrowinds Schedules**

In [1427]:
short_term_loans_schedules = balance_sheet.calculate_short_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=capital_repayment_on_borrowings_df,
    opening_balances=opening_balances,
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

short_term_loans_schedules

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,50000,50000,50000,50000,20000,20000,-100383,499617,374617,374617,874617,824617
Borrowings,0,0,0,0,0,0,600000,0,500000,500000,0,0
Repayments,0,0,0,-30000,0,-120383,0,-125000,-500000,0,-50000,-450000
Closing Balance,50000,50000,50000,20000,20000,-100383,499617,374617,374617,874617,824617,374617


In [1428]:
capital_repayment_on_borrowings_df

,short_term_borrowing,long_term_borrowing,total
Jan-2023,0,0,0
Feb-2023,0,0,0
Mar-2023,0,0,0
Apr-2023,30000,0,30000
May-2023,0,0,0
Jun-2023,120383,0,120383
Jul-2023,0,0,0
Aug-2023,125000,0,125000
Sep-2023,500000,0,500000
Oct-2023,0,0,0


In [1429]:
long_term_loans_schedules = balance_sheet.calculate_long_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=capital_repayment_on_borrowings_df,
    opening_balances=opening_balances,
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

long_term_loans_schedules

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,200000,200000,200000,200000,200000,200000,200000,800000,800000,1300000,1800000,1800000
Borrowings,0,0,0,0,0,0,600000,0,500000,500000,0,0
Repayments,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,200000,200000,200000,200000,200000,200000,800000,800000,1300000,1800000,1800000,1800000


### **Insert Borrowings and Loans**


In [1430]:
balance_sheet_df.loc["Loans"] = long_term_loans_schedules.loc['Closing Balance']
balance_sheet_df.loc["Borrowings"] = short_term_loans_schedules.loc['Closing Balance']

### **Generate Receivables and Payables Schedules**

In [1431]:
trade_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["TRADE_RECEIVABLES"].iat[0],
    receipts_from_receivables=parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
    new_receivables=parameters.loc["NEW_TRADE_RECEIVABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

other_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["OTHER_RECEIVABLES"].iat[0],
    receipts_from_receivables=parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
    new_receivables=parameters.loc["NEW_OTHER_RECEIVABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

intergroup_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["INTERGROUP_RECEIVABLES"].iat[0],
    receipts_from_receivables=parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
    new_receivables=parameters.loc["NEW_INTERGROUP_RECEIVABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

trade_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["TRADE_PAYABLES"].iat[0],
    payments_to_payables=parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
    new_payables=parameters.loc["NEW_TRADE_PAYABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

other_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["OTHER_PAYABLES"].iat[0],
    payments_to_payables=parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
    new_payables=parameters.loc["NEW_OTHER_PAYABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)


balance_sheet_df.loc["Trade Payables"] = trade_payables_schedule.loc["Closing Balance"]
balance_sheet_df.loc["Other Payables"] = other_payables_schedule.loc["Closing Balance"]

balance_sheet_df.loc["Trade Receivables"] = trade_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Other Receivables"] = other_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Intergroup Receivables"] = intergroup_receivables_schedule.loc[
    "Closing Balance"
]


balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"96,000.00","72,000.00","48,000.00","24,000.00",0.00,"11,500.00","-7,000.00","-25,500.00","1,000.00","-22,500.00","-41,000.00","-62,000.00"
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1432]:
balance_sheet_df.loc['Deferred Taxation'] = tax_schedule.loc['Closing Balance']

In [1433]:
inventories_schedule = balance_sheet.generate_inventories_schedule(
    opening_inventories=opening_balances["INVENTORIES"].iat[0],
    new_inventories=parameters.loc["NEW_INVENTORY"],
    inventories_used=parameters.loc["INVENTORY_USED"],
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

inventories_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0,0,0,0,0,0,0,0,0,0,0
New Inventories,0,0,0,0,0,0,0,0,0,0,0,0
Inventories Used,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,0,0,0,0,0,0,0,0,0,0,0,0


In [1434]:
balance_sheet_df.loc['Inventories']  = inventories_schedule.loc['Closing Balance']

### **Equity and Intercompany Loans**


In [1435]:
balance_sheet_df.loc["Issued Share Capital"] = helper.change_period_index_to_strftime(
    parameters.loc["SHARE_CAPITAL"].cumsum()
    + opening_balances["ISSUED_SHARE_CAPITAL"].iat[0]
)

balance_sheet_df.loc["Intercompany Loans"] = helper.change_period_index_to_strftime(
    parameters.loc["INTERCOMPANY_LOANS"].cumsum()
    + opening_balances["INTERCOMPANY_LOANS"].iat[0]
)


balance_sheet_df.loc["Share Premium"] = helper.change_period_index_to_strftime(
    parameters.loc["SHARE_PREMIUM"].cumsum() + opening_balances["SHARE_PREMIUM"].iat[0]
)

balance_sheet_df.loc[
    "Other Components Of Equity"
] = helper.change_period_index_to_strftime(
    parameters.loc["OTHER_COMPONENTS_OF_EQUITY"].cumsum()
    + opening_balances["OTHER_COMPONENTS_OF_EQUITY"].iat[0]
)

balance_sheet_df.loc["Treasury Shares"] = helper.change_period_index_to_strftime(
    parameters.loc["TREASURY_SHARES"].cumsum()
    + opening_balances["TREASURY_SHARES"].iat[0]
)

balance_sheet_df.loc["Retained Earnings"] = (
    income_statement_df.loc["PROFIT/(LOSS) FOR PERIOD"]
    - helper.change_period_index_to_strftime(parameters.loc["DIVIDEND_PAID"])
).cumsum()

balance_sheet_df.loc["Capital And Reserves"] = balance_sheet_df.loc[
    "Issued Share Capital":"Retained Earnings"
].sum()

### **Other Assets**

In [1436]:
balance_sheet_df.loc["Intangible Assets"] = helper.change_period_index_to_strftime(
    parameters.loc["INTANGIBLE_ASSETS"].cumsum()
    + opening_balances["INTANGIBLE_ASSETS"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Subsidiaries"
] = helper.change_period_index_to_strftime(
    parameters.loc["INVESTMENT_IN_SUBSIDIARIES"].cumsum()
    + opening_balances["INVESTMENT_IN_SUBSIDIARIES"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Associates"
] = helper.change_period_index_to_strftime(
    parameters.loc["INVESTMENT_IN_ASSOCIATES"].cumsum()
    + opening_balances["INVESTMENT_IN_ASSOCIATES"].iat[0]
)

balance_sheet_df.loc["Investment Properties"] = helper.change_period_index_to_strftime(
    parameters.loc["INVESTMENT_PROPERTIES"].cumsum()
    + opening_balances["INVESTMENT_PROPERTIES"].iat[0]
)

balance_sheet_df.loc["Equity Investments"] = helper.change_period_index_to_strftime(
    parameters.loc["EQUITY_INVESTMENTS"].cumsum()
    + opening_balances["EQUITY_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Long Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    parameters.loc["LONG_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["LONG_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Short Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    parameters.loc["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Loans To Related Entities"
] = helper.change_period_index_to_strftime(
    parameters.loc["LOANS_TO_RELATED_ENTITIES"].cumsum()
    + opening_balances["LOANS_TO_RELATED_ENTITIES"].iat[0]
)

In [1437]:
balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"96,000.00","72,000.00","48,000.00","24,000.00",0.00,"11,500.00","-7,000.00","-25,500.00","1,000.00","-22,500.00","-41,000.00","-62,000.00"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,0.00,0.00,0.00,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00"
Investment In Associates,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00"
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,000.00","5,000.00","5,000.00","5,000.00"
Equity Investments,"20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","30,000.00"
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### **Calculate Totals**


In [1440]:
balance_sheet_df = balance_sheet.sum_financial_statements_totals(balance_sheet_df)
balance_sheet_df = balance_sheet.calculate_final_balances(balance_sheet_df=balance_sheet_df)
balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"96,000.00","72,000.00","48,000.00","24,000.00",0.00,"11,500.00","-7,000.00","-25,500.00","1,000.00","-22,500.00","-41,000.00","-62,000.00"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,0.00,0.00,0.00,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00"
Investment In Associates,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00"
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,000.00","5,000.00","5,000.00","5,000.00"
Equity Investments,"20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","30,000.00"
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## **Indirect Cashflow**

In [ ]:
statement_of_cashflow_df = (
    statement_of_cashflows.generate_statement_of_cashflow_template(
        VALUATION_DATE, MONTHS_TO_FORECAST
    )
)

In [ ]:
change_in_trade_payables = trade_payables_schedule.loc['Closing Balance'] - trade_payables_schedule.loc['Opening Balance']
change_in_trade_receivables = trade_receivables_schedule.loc['Opening Balance'] - trade_receivables_schedule.loc['Closing Balance']
change_in_loan_book_principle = loan_book_df.loc['Opening Balance'] - loan_book_df.loc['Closing Balance'] 


### **Inserting Readily Available Figures**

In [ ]:
statement_of_cashflow_df.loc['Profit/(loss) per I/S'] = income_statement_df.loc['PROFIT/(LOSS) FOR PERIOD']
statement_of_cashflow_df.loc['Depreciation'] = income_statement_df.loc['Depreciation']
statement_of_cashflow_df.loc['(Increase)/Decrease in Receivables'] = change_in_trade_receivables
statement_of_cashflow_df.loc['Increase/(Decrease) in Payables'] = change_in_trade_payables
statement_of_cashflow_df.loc['(Increase)/Decrease in Loan Book (Principle)'] = change_in_loan_book_principle
statement_of_cashflow_df.loc['Dividend Paid'] = helper.change_period_index_to_strftime(parameters.loc["DIVIDEND_PAID"])
statement_of_cashflow_df.loc['Interest Paid'] = finance_costs.loc['total']
statement_of_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']
statement_of_cashflow_df.loc['Repayment of Borrowings'] = capital_repayment_on_borrowings_df['total']
statement_of_cashflow_df.loc['Purchase of Fixed Assets'] = capital_expenses
statement_of_cashflow_df.loc["Increase/(Decrease) in Short Term Borrowings"] = (
    short_term_loans_schedules.loc["Closing Balance"]
    - short_term_loans_schedules.loc["Opening Balance"]
)

statement_of_cashflow_df.loc["Increase/(Decrease) in Long Term Borrowings"] = (
    long_term_loans_schedules.loc["Closing Balance"]
    - long_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df = balance_sheet.sum_financial_statements_totals(statement_of_cashflow_df)
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df.fillna(0, inplace=True)

In [ ]:
statement_of_cashflow_df

### **Calculating Totals**

In [ ]:
statement_of_cashflow_df.loc['Net Increase/(Decrease) in Cash'] = (
    statement_of_cashflow_df.loc["Net Cash Flow From Operations"]
    + statement_of_cashflow_df.loc["Cash Flow From Investing Activities"]
    + statement_of_cashflow_df.loc["Cash Flow From Financing Activities"]
)

In [ ]:
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df = (
    statement_of_cashflows.calculate_cash_at_end_and_beginning_of_period(
        statement_of_cashflow_df=statement_of_cashflow_df,
        opening_balances=opening_balances,
    )
)
statement_of_cashflow_df